In [75]:
from polygon_backfill import get_open_market_dates, dates_from_s3, find_remaining_dates, get_ticks_date_df, backfill_date

symbols = ['GLD', 'GDX', 'SPY', 'GOLD']
start_date = '2020-01-01'
end_date = '2020-10-20'
tick_type = 'trades'
result_path = '/Users/bobcolner/QuantClarity/pandas-polygon/data'

In [4]:
import ray
ray.init(
    num_cpus=5,
    ignore_reinit_error=True,
    )

@ray.remote
def backfill_date_ray(symbol, date, tick_type, result_path):
    result = backfill_date(
        symbol=symbol, 
        date=date, 
        tick_type=tick_type, 
        result_path=result_path,
        save_local=True,
        upload_to_s3=True,
    )
    return symbol + '|' + date


2020-10-19 17:52:42,923	ERROR worker.py:643 -- Calling ray.init() again after it has already been called.


In [ ]:
request_dates = get_open_market_dates(start_date, end_date)
futures = []
for symbol in symbols: 
    existing_dates = dates_from_s3(symbol, tick_type)
    remaining_dates = find_remaining_dates(request_dates, existing_dates)
    for date in remaining_dates:
        result = backfill_date_ray.remote(
            symbol=symbol, 
            date=date, 
            tick_type=tick_type, 
            result_path=result_path
        )
        futures.append(result)
        
ray.get(futures)